This is version 2 of my Fantasy Hockey Analyzer. The purpose of this notebook is to predict the number of fantasy points every hockey player in the league will get based on previous years' performance.

This notebook primarily uses data from moneypuck.com for analysis, and it also uses data from rotowire.com to get +/- for each player.

Section 1: Parameters and Modules

These are the variables that can be adjusted. My model is an ensemble model consisting of neural nets and random forests, with data going back one, two, and three years.

In [1]:
# Set these values to the appropriate ammonts

current_year = 2025
common_number = 1
number_of_one_year_neural_nets = common_number
number_of_two_year_neural_nets = common_number
number_of_three_year_neural_nets = common_number
number_of_one_year_random_forests = common_number
number_of_two_year_random_forests = common_number
number_of_three_year_random_forests = common_number

# This is the breakdown of how many fantasy points a player gets for each category
points_dictionary = {
    'Goals':5, 
    'Assists':3, 
    '+/-':1.5, 
    'PIM':-0.25, 
    'PP_Goals':4, 
    'PP_Assists':2, 
    'SH_Goals':6, #won't count SHG from 5-on-3
    'SH_Assists':4, 
    'Faceoffs_Won':0.25, 
    'Faceoffs_Lost':-0.15, 
    'Hits':0.5, 
    'Blocked_Shots':0.75
    }

The following is a list of modules that I used and the reason why they were used:

-os: to allow the program to read data in the repository

-numpy: basic math operations

-pandas: all dataframe operations/data storage/data cleaning

-various sklearn: all machine learning operations/analysis

In addition to these modules, I also have a custom module that contains helper functions that help in data cleaning/accuracy evaluation. These functions are contained in the "my_module.py" file in the repository. If you are interested in taking a look at these functions, they are available at https://github.com/chrisberry888/FantasyHockeyAnalyzer in the "my_module.py" file.

In [2]:
#Import block
import os
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.base import clone
import my_module_v2 as mx

pd.set_option('display.max_columns', None)

Section 2: Data Gathering and Cleaning

This section compiles the Moneypuck and Rotowire data into a format that is usable by the ML models.

There is some discrepencies between stats on ESPN and on moneypuck. These shouldn't alter the fantasy points too much. For example, Sydney Crosby is short-changed 1 faceoff win but has one additional hit in moneypuck than on ESPN, so in my model he has 0.10 more points than he does on ESPN. I would like to see why this is the case in the future/get fully accurate data (perhaps from ESPN themselves), but for now I am ok with this very small error.

In [3]:
yearly_player_data = []

for year in range(2010, current_year):
    moneypuck_data = mx.get_moneypuck_data(year)
    rotowire_data = mx.get_rotowire_data(year)
    combined_data = mx.combine_dataframes(moneypuck_data, rotowire_data)
    final_df = mx.calculate_fantasy_points(combined_data, points_dictionary)
    
    yearly_player_data.append(final_df)

    

This next cell compiles the yearly data into chunks of one, two, and three-year data to be used by the ML models.

In [4]:
ml_data_one_year = mx.get_ml_data(yearly_player_data, current_year, 1)
ml_data_two_years = mx.get_ml_data(yearly_player_data, current_year, 2)
ml_data_three_years = mx.get_ml_data(yearly_player_data, current_year, 3)

The data is now ready to be used to train the ML model.

In [5]:

one_year_X, one_year_y = mx.separate_fantasy_points(ml_data_one_year)
two_year_X, two_year_y = mx.separate_fantasy_points(ml_data_two_years)
three_year_X, three_year_y = mx.separate_fantasy_points(ml_data_three_years)

In [6]:
ml_data_three_years[ml_data_three_years['season'] == 2021].head()

playerId  season  games_played  4on5_icetime  5on4_icetime  \
3525   8470600    2021            82        7753.0       10123.0   
3526   8470613    2021            82       13145.0       12410.0   
3527   8471685    2021            81        7932.0       15210.0   
3528   8471709    2021            75        3668.0         170.0   
3529   8471724    2021            78        6326.0       15204.0   

      5on5_icetime  all_icetime  other_icetime  4on5_shifts  5on4_shifts  \
3525       93680.0     116385.0         4829.0        169.0        180.0   
3526       96721.0     128642.0         6366.0        264.0        177.0   
3527       73035.0     100940.0         4763.0        228.0        174.0   
3528       63830.0      68469.0          801.0        113.0         14.0   
3529       92780.0     120638.0         6328.0        155.0        153.0   

      5on5_shifts  all_shifts  other_shifts  4on5_gameScore  5on4_gameScore  \
3525       1763.0      2185.0          73.0           31.10           32.72   
3526       1920.0      2441.0          80.0           38.49           40.91   
3527       1483.0      1980.0          95.0           76.39           77.22   
3528       1454.0      1598.0          17.0            1.53            2.38   
3529       1772.0      2147.0          67.0           63.75           68.72   

      5on5_gameScore  all_gameScore  other_gameScore  \
3525           34.50          34.50            26.90   
3526           41.61          41.61            29.41   
3527           77.22          77.22            64.98   
3528            2.92           2.92             1.35   
3529           70.85          70.85            62.60   

      4on5_onIce_xGoalsPercentage  5on4_onIce_xGoalsPercentage  \
3525                         0.19                         0.91   
3526                         0.09                         0.86   
3527                         0.09                         0.85   
3528                         0.13                         0.64   
3529                         0.16                         0.86   

      5on5_onIce_xGoalsPercentage  all_onIce_xGoalsPercentage  \
3525                         0.52                        0.55   
3526                         0.48                        0.46   
3527                         0.54                        0.55   
3528                         0.44                        0.41   
3529                         0.51                        0.56   

      other_onIce_xGoalsPercentage  4on5_offIce_xGoalsPercentage  \
3525                          0.53                          0.13   
3526                          0.38                          0.21   
3527                          0.58                          0.20   
3528                          0.12                          0.15   
3529                          0.54                          0.10   

      5on4_offIce_xGoalsPercentage  5on5_offIce_xGoalsPercentage  \
3525                          0.86                          0.51   
3526                          0.80                          0.47   
3527                          0.84                          0.53   
3528                          0.83                          0.49   
3529                          0.82                          0.55   

      all_offIce_xGoalsPercentage  other_offIce_xGoalsPercentage  \
3525                         0.52                           0.49   
3526                         0.47                           0.39   
3527                         0.52                           0.55   
3528                         0.47                           0.46   
3529                         0.50                           0.46   

      4on5_onIce_corsiPercentage  5on4_onIce_corsiPercentage  \
3525                        0.18                        0.85   
3526                        0.13                        0.84   
3527                        0.12                        0.89   
3528                        0.19                        0

In [7]:
one_year_neural_nets = mx.create_models(
    one_year_X,
    one_year_y,
    MLPRegressor(max_iter=1000),
    number_of_one_year_neural_nets
)

In [ ]:
one_year_random_forests = mx.create_models(
    one_year_X,
    one_year_y,
    RandomForestRegressor(),
    number_of_one_year_random_forests
)

In [ ]:
two_year_neural_nets = mx.create_models(
    two_year_X,
    two_year_y,
    MLPRegressor(max_iter=1000),
    number_of_two_year_neural_nets
)

In [ ]:
two_year_random_forests = mx.create_models(
    two_year_X,
    two_year_y,
    RandomForestRegressor(),
    number_of_two_year_random_forests
)

In [ ]:
three_year_neural_nets = mx.create_models(
    three_year_X,
    three_year_y,
    MLPRegressor(max_iter=1000),
    number_of_three_year_neural_nets
)

In [ ]:
three_year_random_forests = mx.create_models(
    three_year_X,
    three_year_y,
    RandomForestRegressor(),
    number_of_three_year_random_forests
)

Now we generate the table with final predictions.

In [7]:
current_one_year_X = mx.get_final_year_data(yearly_player_data, 1)
current_two_year_X = mx.get_final_year_data(yearly_player_data, 2)
current_three_year_X = mx.get_final_year_data(yearly_player_data, 3)

In [10]:
current_three_year_X.head()

playerId  season  games_played  4on5_icetime  5on4_icetime  5on5_icetime  \
0   8470600    2022            82        7099.0        7749.0       82700.0   
1   8470613    2022            82        9866.0       13997.0       85320.0   
2   8471677    2022            83        9801.0         246.0       82945.0   
3   8471685    2022            82        8674.0       13828.0       72115.0   
4   8471709    2022            78       12122.0         260.0       67729.0   

   all_icetime  other_icetime  4on5_shifts  5on4_shifts  5on5_shifts  \
0     100267.0         2719.0        193.0        168.0       1612.0   
1     114219.0         5036.0        224.0        183.0       1705.0   
2      95333.0         2341.0        219.0         23.0       1653.0   
3      99887.0         5270.0        226.0        182.0       1471.0   
4      81762.0         1651.0        226.0         23.0       1490.0   

   all_shifts  other_shifts  4on5_gameScore  5on4_gameScore  5on5_gameScore  \
0      2019.0          46.0           33.10           32.30           35.10   
1      2193.0          81.0           94.01           90.86           96.79   
2      1939.0          44.0           -6.09           -0.16           -5.21   
3      1984.0         105.0           72.11           77.45           77.68   
4      1777.0          38.0           14.35            7.15           14.58   

   all_gameScore  other_gameScore  4on5_onIce_xGoalsPercentage  \
0          35.10            22.75                         0.13   
1          96.79            73.19                         0.15   
2          -5.21            -0.71                         0.08   
3          77.68            65.17                         0.04   
4          14.58            11.78                         0.06   

   5on4_onIce_xGoalsPercentage  5on5_onIce_xGoalsPercentage  \
0                         0.89                         0.53   
1                         0.88                         0.62   
2                         0.65                         0.41   
3                         0.90                         0.54   
4                         1.00                         0.51   

   all_onIce_xGoalsPercentage  other_onIce_xGoalsPercentage  \
0                        0.54                          0.46   
1                        0.60                          0.53   
2                        0.36                          0.37   
3                        0.55                          0.52   
4                        0.44                          0.47   

   4on5_offIce_xGoalsPercentage  5on4_offIce_xGoalsPercentage  \
0                          0.15                          0.90   
1                          0.18                          0.90   
2                          0.18                          0.79   
3                          0.16                          0.90   
4                          0.08                          0.89   

   5on5_offIce_xGoalsPercentage  all_offIce_xGoalsPercentage  \
0                          0.52                         0.55   
1                          0.59                         0.57   
2                          0.47                         0.51   
3                          0.53                         0.52   
4                          0.50                         0.50   

   other_offIce_xGoalsPercentage  4on5_onIce_corsiPercentage  \
0                           0.59                        0.12   
1                           0.54                        0.20   
2                           0.52                        0.10   
3                           0.56                        0.07   
4                           0.46                        0.08   

   5on4_onIce_corsiPercentage  5on5_onIce_corsiPercentage  \
0                        0.83                        0.53   
1                        0.88                        0.62   
2                        0.83                        0.42   
3                        0.88                        0.52   